# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [4]:
# Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-5-nano'
# openai = OpenAI()

MODEL = "llama3.3"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [5]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [8]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [9]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [10]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company website', 'url': 'https://edwarddonner.com/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [11]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [14]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [15]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
moonshotai/Kimi-K2-Thinking
Updated
5 days ago
•
105k
•
1.12k
maya-research/maya1
Updated
1 day ago
•
18.6k
•
539
baidu/ERNIE-4.5-VL-28B-A3B-Thinking
Updated
1 day ago
•
4.41k
•
344
dx8152/Qwen-Edit-2509-Multiple-angles
Updated
1 day ago
•
40.4k
•
554
MiniMaxAI/MiniMax-M2
Updated
about 5 hours ago
•
891k
•
1.28k
Browse 1M+ models
Spaces
Running
on
Zero
713
713
Qwen Image Edit Camera Control
🎬
Fast 4 step inference with Qwen Image Edit 2509
Running
on
CPU Upgrade
2.13k
2.13k
The Smol Training Playbook
📚
The secrets to building world-class LLMs
Running
on
Zero
365
365
Qwen-Image-2509-MultipleAngles
👀
Qwen-Image-2509-MultipleAngles


In [16]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

"\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Thinking\nUpdated\n5 days ago\n•\n105k\n•\n1.12k\nmaya-research/maya1\nUpdated\n1 day ago\n•\n18.6k\n•\n539\nbaidu/ERNIE-4.5-VL-28B-A3B-Thinking\nUpdated\n1 day ago\n•\n4.41k\n•\n344\ndx8152/Qwen-Edit-2509-Multiple-angles\nUpdated\n1 day ago\n•\n40.4k\n•\n554\nMiniMaxAI/MiniMax-M2\nUpdated\nabout 5 hours ago\n•\n891k\n•\n1.28k\nBrowse 1M+ models\nSpaces\nRunning\non\nZero

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

# Introduction to Hugging Face
Hugging Face is a collaboration platform for the machine learning community, empowering users to create, discover, and collaborate on ML models, datasets, and applications. The company is at the heart of the AI revolution, with a fast-growing community, open-source ML libraries and tools, and a talented science team exploring the edge of technology.

## Company Culture
Hugging Face values collaboration, openness, and ethics in AI development. The company believes in building an open and ethical AI future together, where anyone can share, explore, discover, and experiment with open-source ML. The Hugging Face Hub is a central place for the machine learning community to come together and drive innovation.

## Customers
Hugging Face serves a wide range of customers, from individual machine learning engineers and scientists to enterprises and organizations. The company provides paid Compute and Enterprise solutions to accelerate ML development and deployment. With over 1 million models, 400k+ applications, and 250k+ datasets available on the platform, Hugging Face is the go-to destination for anyone looking to build, deploy, or collaborate on machine learning projects.

## Careers
Hugging Face is committed to fostering a culture of innovation and collaboration. The company is always looking for talented individuals to join its team, from software engineers and data scientists to researchers and designers. With a strong focus on open-source development and community engagement, Hugging Face offers a unique opportunity for professionals to contribute to the future of AI and machine learning.

## Community
The Hugging Face community is at the core of the company's mission. With a vast array of models, datasets, and applications available, users can explore, learn, and collaborate on various projects. The company also hosts a blog, forum, and social media channels to facilitate discussion, knowledge sharing, and community engagement.

## Products and Services
Hugging Face offers a range of products and services, including:

* Models: Over 1 million pre-trained models available for use
* Datasets: A vast collection of datasets for various machine learning tasks
* Spaces: A platform for running and deploying machine learning applications
* Enterprise: Paid solutions for enterprises and organizations to accelerate ML development and deployment
* Compute: Cloud-based infrastructure for running and training machine learning models

Overall, Hugging Face is a pioneer in the machine learning community, driving innovation and collaboration in AI development. With its strong focus on openness, ethics, and community engagement, the company is poised to shape the future of artificial intelligence and machine learning.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="llama3.3",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Introduction to Hugging Face
Hugging Face is a pioneering AI community that is building the future of machine learning. The company provides a collaborative platform where developers, researchers, and organizations can host, share, and work together on models, datasets, and applications.

## Our Mission
The mission of Hugging Face is to empower the machine learning community to create, discover, and collaborate on innovative projects. With over 1 million models, 250 thousand datasets, and 400 thousand applications, the platform has become the go-to destination for AI enthusiasts and professionals alike.

## Company Culture
At Hugging Face, we value collaboration, open-source innovation, and community-driven development. Our team is passionate about fostering a culture of creativity, inclusivity, and continuous learning. We believe that by working together and sharing knowledge, we can accelerate the development of AI solutions that benefit society as a whole.

## Customers and Community
Our platform serves a diverse range of customers, from individual researchers to large enterprises. The Hugging Face community is comprised of developers, data scientists, and researchers who are passionate about machine learning and eager to contribute to the growth of the field. With a strong focus on open-source collaboration, we have established partnerships with leading organizations and institutions to drive innovation and adoption of AI technologies.

## Careers at Hugging Face
We are always looking for talented individuals to join our team. Our careers page lists current openings in areas such as software development, research, marketing, and sales. If you are passionate about AI, machine learning, and collaboration, we encourage you to explore opportunities with us.

## News and Insights
Our blog features articles, guides, and discussions on the latest developments in machine learning, including NLP, audio, computer vision, reinforcement learning, and more. We also share case studies, community articles, and research papers to provide insights into the applications and potential of AI technologies.

## Join Us
Whether you are a developer looking for resources, a researcher seeking collaboration opportunities, or an organization interested in leveraging AI solutions, we invite you to join the Hugging Face community today. Sign up for our platform, explore our models and datasets, and discover how we can help accelerate your machine learning journey.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>